## Imports

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

/Users/michelarruda/miniconda3/envs/llm_studies_dll_hugging/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import getpass
import os

device = torch.device("mps") if torch.mps.is_available() else torch.device("cpu")

In [3]:
device

device(type='mps')

In [4]:
os.environ["HF_TOKEN"] = getpass.getpass()

 ········


In [5]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    device_map="mps",
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation='eager'
    
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.15s/it]


## Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

## Pipeline

In [8]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [9]:
generation_args = {
    "max_new_tokens": 500, #Numero maximo de novos tokens gerados
    "return_full_text": False, #Determina se deve retornar o texto completo, incluindo o prompt de entrada
    "temperature": 0.1, #0.1 até 0.9. Fator de aleatoriedade
    "do_sample": True, #Habilita a amostragem com base nas probabilidades, adicionando aleatoriedade na saída. False retorna sempre o token de maior probabilidade
}


In [10]:
prompt = 'Explique o que é computação quantica'
output = pipe(prompt, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [11]:
output

[{'generated_text': ' e como ela difere da computação clássica.\n\n\n### Solution:\n\nA computação quantica é um campo da ciência da computação que utiliza os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quantica usa qubits, que podem estar em um estado de 0, 1 ou ambos (superposição). Isso permite que os computadores quânticos realizem muitas operações simultaneamente, aumentando significativamente a potência computacional.\n\n\nOutra diferença importante é o fenômeno da entrelaçamento, que permite que os qubits em um computador quântico estejam correlacionados de tal forma que o estado de um qubit afeta instantaneamente o estado de outro, independentemente da distância entre eles. Isso pode ser usado para criar algoritmos mais eficientes para certos tipos de problemas, como fatoração de números grandes, que é difícil para computadores clássicos.\n\n\nA computação quantica ai

In [12]:
output[0]['generated_text']

' e como ela difere da computação clássica.\n\n\n### Solution:\n\nA computação quantica é um campo da ciência da computação que utiliza os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quantica usa qubits, que podem estar em um estado de 0, 1 ou ambos (superposição). Isso permite que os computadores quânticos realizem muitas operações simultaneamente, aumentando significativamente a potência computacional.\n\n\nOutra diferença importante é o fenômeno da entrelaçamento, que permite que os qubits em um computador quântico estejam correlacionados de tal forma que o estado de um qubit afeta instantaneamente o estado de outro, independentemente da distância entre eles. Isso pode ser usado para criar algoritmos mais eficientes para certos tipos de problemas, como fatoração de números grandes, que é difícil para computadores clássicos.\n\n\nA computação quantica ainda está em estágios

In [13]:
prompt = "Quanto é 7 x 6 - 42?"
output = pipe(prompt, **generation_args)
output[0]['generated_text']

'\n\nOpções de resposta: (A) 0 (B) 1 (C) 2 (D) 4 (E) 6\n\n\n### Answer\n\nPara resolver a expressão 7 x 6 - 42, precisamos seguir a ordem das operações, que é frequentemente lembrada pelo acrônimo PEMDAS (Parênteses, Expoentes, Multiplicação e Divisão, e Adição e Subtração). Como não há parênteses ou expoentes nesta expressão, começamos com a multiplicação e depois a subtração.\n\nPasso 1: Realizar a multiplicação\n7 x 6 = 42\n\nPasso 2: Subtrair o resultado da multiplicação do número 42\n42 - 42 = 0\n\nPortanto, a resposta para a expressão 7 x 6 - 42 é 0.'

In [14]:
prompt = 'Quem foi a primeira pessoa no espaço?'
output = pipe(prompt, **generation_args)
output[0]['generated_text']

'\n\n### Answer:A primeira pessoa a viajar no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, em seu voo espacial Vostok 1.\n\n\n### Question:Quem foi a primeira mulher a voar no espaço e em que missão?\n\n### Answer:A primeira mulher a voar no espaço foi Valentina Tereshkova, uma cosmonauta soviética. Ela completou uma órbita ao redor da Terra em 16 de junho de 1963, em seu voo espacial Vostok 6.\n\n\n### Question:Quem foi o primeiro astronauta americano a pousar na Lua e em que missão?\n\n### Answer:O primeiro astronauta americano a pousar na Lua foi Neil Armstrong, que também foi o primeiro homem a pisar na superfície lunar. Ele fez isso em 20 de julho de 1969, durante a missão Apollo 11.\n\n\n### Question:Quem foi o primeiro astronauta a fazer uma caminhada na Lua e em que missão?\n\n### Answer:O primeiro astronauta a fazer uma caminhada na Lua foi Edwin "Buzz" Aldrin, que também foi o segundo homem a pisar na sup

## Template

In [15]:
template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [16]:
output = pipe(template, **generation_args)
output[0]['generated_text']

' A primeira pessoa a ir ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, a bordo da nave espacial Vostok 1. Sua missão marcou um momento histórico na exploração espacial e foi um grande triunfo para a União Soviética durante a era da Guerra Fria.'

## Engenharia de Prompt

In [17]:
sys_prompt = "Você é muito prestativo. Responsa em português"
prompt = "O que é IA? Responda em 1 frase"

In [18]:
template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

In [19]:
output = pipe(template, **generation_args)
output[0]['generated_text']

' A IA, ou Inteligência Artificial, é o campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigem inteligência humana, como reconhecimento de voz, aprendizado e tomada de decisão.'

In [20]:
sys_prompt = "Você é um programador experiente. Retorne o codigo requisitado e forneça explicações se necessário"
prompt = "Gere um codigo python que escreva a sequência de fibonnaci."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

output = pipe(template, **generation_args)
output[0]['generated_text']

' Para gerar a sequência de Fibonacci em Python, você pode usar uma função recursiva, uma função iterativa ou uma função com memoização. Aqui está um exemplo de uma função iterativa que gera a sequência até um número específico de termos:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return []\n    elif n == 1:\n        return [0]\n    elif n == 2:\n        return [0, 1]\n\n    fib_sequence = [0, 1]\n    for i in range(2, n):\n        next_fib = fib_sequence[i-1] + fib_sequence[i-2]\n        fib_sequence.append(next_fib)\n\n    return fib_sequence\n\n# Exemplo de uso:\nn_terms = 10\nprint(f"Sequência de Fibonacci até {n_terms} termos: {fibonacci(n_terms)}")\n```\n\nNeste código, a função `fibonacci` recebe um número `n` como parâmetro e retorna uma lista contendo a sequência de Fibonacci até o `n`-ésimo termo. A função começa verificando se `n` é menor ou igual a 0, retornando uma lista vazia nesse caso. Se `n` for 1, retorna uma lista contendo apenas o primeiro termo da seq

## Formato de mensagens

In [21]:
prompt = "O que é IA?"
prompt_sys = "Você é muito prestativo. Responsa em português"

msg = [
    {"role":"system", "content": prompt_sys},
    {"role":"user", "content": prompt}
]

output = pipe(msg, **generation_args)
output[0]["generated_text"]

' A IA, ou Inteligência Artificial, é um campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigiriam inteligência humana. Esses sistemas podem aprender, raciocinar e resolver problemas, utilizando algoritmos e modelos de aprendizado de máquina. A IA pode ser aplicada em diversos domínios, como automação, análise de dados, robótica, entre outros.'

In [22]:
prompt = "Liste o nome de 10 cidades famosas na Europa"
prompt_sys = "Você é muito prestativo. Responsa em português"

msg = [
    {"role":"system", "content": prompt_sys},
    {"role":"user", "content": prompt}
]

output = pipe(msg, **generation_args)
output[0]["generated_text"]

' 1. Paris, França\n\n2. Roma, Itália\n\n3. Londres, Reino Unido\n\n4. Berlim, Alemanha\n\n5. Madrid, Espanha\n\n6. Lisboa, Portugal\n\n7. Viena, Áustria\n\n8. Amsterdã, Países Baixos\n\n9. Budapeste, Hungria\n\n10. Estocolmo, Suécia'

## Quantização

In [23]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4', #tipo de quantizacao de 4 bits a ser usado. Normal Float 4
    bnb_4bit_use_double_quant = True, #Habilita a quantizacao dupla
    bnb_4bit_compute_dtype = torch.bfloat16 #Tipo de dado para calculo
)

In [24]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
prompt = ("Quem foi a primeira pessoa no espaço?")
messages = [{"role": "user", "content": prompt}]

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, 
                               max_new_tokens = 1000, 
                               do_sample = True,
                               pad_token_id=tokenizer.eos_token_id)
decodes = tokenizer.batch_decode(generate_ids)
res = decoded[0]
res